# Tuning

We obtained 72.87% accuracy in AlphabetSoupCharity, but our goal is 75% accuracy. Ideally, we want to tune the neural network parameters to get better accuracy, because the neural network is what we're all here for.

So, we're going to use keras-tuner to do its best to find the most effective neural network parameters.

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
#application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df = pd.read_csv("Data/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], axis=1)

In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df["APPLICATION_TYPE"].value_counts().index[8:]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df["CLASSIFICATION"].value_counts().index[5:]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [6]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [7]:
# Split our preprocessed data into our features and target arrays
features = application_df.drop("IS_SUCCESSFUL", axis=1)
target = application_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=1)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train and Evaluate the Model

In [9]:
num_features = len(features.columns)

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    model = tf.keras.models.Sequential()

    num_layers = hp.Int('num_layers', 1, 5)

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation_funcs = ['relu', 'selu', 'gelu', 'tanh', 'sigmoid']
    activation = hp.Choice('activation_0',activation_funcs)
    
    # Allow kerastuner to decide number of neurons in first layer
    model.add(tf.keras.layers.Dense(units=hp.Int('units_0',
        min_value=1,
        max_value=10,
        step=1), activation=activation, input_dim=num_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(num_layers):
        activation = hp.Choice(f'activation_{i+1}',activation_funcs)
        model.add(tf.keras.layers.Dense(units=hp.Int(f'units_{i+1}',
            min_value=1,
            max_value=30,
            step=1),
            activation=activation))
    
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return model

In [10]:
# Import the kerastuner library
import keras_tuner as kt

# Create a `Hyperband()` tuner instance
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=100,
    hyperband_iterations=2,
    #directory=os.path.normpath("C:/")
    )

Using TensorFlow backend


In [11]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 508 Complete [00h 03m 24s]
val_accuracy: 0.7325947284698486

Best val_accuracy So Far: 0.7342274188995361
Total elapsed time: 02h 37m 19s


In [14]:
# Get top 3 model hyperparameters and print the values
best_hypers = tuner.get_best_hyperparameters(10)
for best_hyper in best_hypers:
    print(best_hyper.values)

{'num_layers': 2, 'activation_0': 'selu', 'units_0': 6, 'activation_1': 'selu', 'units_1': 13, 'activation_2': 'tanh', 'units_2': 7, 'activation_3': 'sigmoid', 'units_3': 30, 'activation_4': 'tanh', 'units_4': 7, 'activation_5': 'gelu', 'units_5': 13, 'tuner/epochs': 34, 'tuner/initial_epoch': 12, 'tuner/bracket': 4, 'tuner/round': 3, 'tuner/trial_id': '0132'}
{'num_layers': 1, 'activation_0': 'relu', 'units_0': 4, 'activation_1': 'gelu', 'units_1': 23, 'activation_2': 'relu', 'units_2': 10, 'activation_3': 'tanh', 'units_3': 4, 'activation_4': 'tanh', 'units_4': 27, 'activation_5': 'relu', 'units_5': 22, 'tuner/epochs': 34, 'tuner/initial_epoch': 12, 'tuner/bracket': 4, 'tuner/round': 3, 'tuner/trial_id': '0394'}
{'num_layers': 5, 'activation_0': 'relu', 'units_0': 8, 'activation_1': 'sigmoid', 'units_1': 19, 'activation_2': 'relu', 'units_2': 7, 'activation_3': 'sigmoid', 'units_3': 23, 'activation_4': 'sigmoid', 'units_4': 6, 'activation_5': 'relu', 'units_5': 24, 'tuner/epochs': 10

In [15]:
# Evaluate the top 3 models against the test dataset
best_models = tuner.get_best_models(10)
for best_model in best_models:
    model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5524 - accuracy: 0.7342 - 454ms/epoch - 2ms/step
Loss: 0.5523501634597778, Accuracy: 0.7342274188995361
268/268 - 0s - loss: 0.5554 - accuracy: 0.7339 - 440ms/epoch - 2ms/step
Loss: 0.555363118648529, Accuracy: 0.7338775396347046
268/268 - 0s - loss: 0.5538 - accuracy: 0.7339 - 492ms/epoch - 2ms/step
Loss: 0.5537852048873901, Accuracy: 0.7338775396347046
268/268 - 0s - loss: 0.5519 - accuracy: 0.7338 - 443ms/epoch - 2ms/step
Loss: 0.551945149898529, Accuracy: 0.7337609529495239
268/268 - 0s - loss: 0.5542 - accuracy: 0.7335 - 441ms/epoch - 2ms/step
Loss: 0.5541790127754211, Accuracy: 0.7335277199745178
268/268 - 0s - loss: 0.5568 - accuracy: 0.7335 - 492ms/epoch - 2ms/step
Loss: 0.5568428039550781, Accuracy: 0.7335277199745178
268/268 - 0s - loss: 0.5546 - accuracy: 0.7335 - 432ms/epoch - 2ms/step
Loss: 0.5546294450759888, Accuracy: 0.7335277199745178
268/268 - 0s - loss: 0.5549 - accuracy: 0.7333 - 443ms/epoch - 2ms/step
Loss: 0.5548518896102905, Accuracy: 0.733

# Analysis

Given the fairly-consistent accuracies across the top 10 best tunings using fairly different combinations of layers, neurons, and activation functions, we can surmise that ~73.5% is the upper limit of accuracy when modifying the neural network directly. Other gains will have to come from tweaking the features. See the FeatureTweaks notebook here.